In [11]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as sco
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Nifty 50 Stocks List
nifty50_tickers = [
    "RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "INFY.NS", "ICICIBANK.NS", 
    "HINDUNILVR.NS", "SBIN.NS", "ITC.NS", "BHARTIARTL.NS", "KOTAKBANK.NS",
    "AXISBANK.NS", "BAJFINANCE.NS", "SUNPHARMA.NS", "BAJAJFINSV.NS", "ADANIPORTS.NS",
    "MARUTI.NS", "M&M.NS", "POWERGRID.NS", "NTPC.NS", "DRREDDY.NS",
    "CIPLA.NS", "COALINDIA.NS", "JSWSTEEL.NS", "TITAN.NS", 
    "ULTRACEMCO.NS", "LT.NS", "TECHM.NS", "NESTLEIND.NS", "GRASIM.NS",
    "ASIANPAINT.NS", "SHREECEM.NS", "UPL.NS", "BAJAJ-AUTO.NS", "AMBUJACEM.NS",
    "INDUSINDBK.NS", "BRITANNIA.NS", "DABUR.NS", "WIPRO.NS",
    "ZOMATO.NS", "PIDILITIND.NS", "DIVISLAB.NS", "CHOLAFIN.NS", "ADANIENT.NS",
    "TATACONSUM.NS", "ADANIGREEN.NS", "SAIL.NS", "GODREJCP.NS"
]

class EnhancedPortfolioOptimizer:
    def __init__(self, tickers, start_date="2020-01-01", end_date="2025-01-01", 
                 risk_free_rate=0.07, momentum_window=126, sentiment_factor=0.08):
        """
        Enhanced Portfolio Optimizer with advanced analytics
        
        Parameters:
        - tickers: List of stock tickers
        - start_date/end_date: Analysis period
        - risk_free_rate: Risk-free rate (higher for emerging markets like India)
        - momentum_window: Days to consider for momentum calculation
        - sentiment_factor: Factor to adjust returns based on market sentiment
        """
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.risk_free_rate = risk_free_rate
        self.momentum_window = momentum_window
        self.sentiment_factor = sentiment_factor
        
        # Get market index for reference
        self.market_index = yf.download("^NSEI", start=start_date, end=end_date, progress=False)
        
        # Fetch and process stock data
        self.fetch_stock_data()
        self.prepare_analytics()
    
    def fetch_stock_data(self):
        """Get historical data for all valid tickers"""
        valid_tickers = []
        valid_data = pd.DataFrame()
        
        for ticker in self.tickers:
            try:
                stock_data = yf.download(
                    ticker, 
                    start=self.start_date, 
                    end=self.end_date, 
                    progress=False
                )
                
                if not stock_data.empty and len(stock_data) > 100:
                    valid_tickers.append(ticker)
                    valid_data[ticker] = stock_data['Close']
            except Exception as e:
                print(f"Error downloading {ticker}: {e}")
        
        # Update with valid data
        self.tickers = valid_tickers
        self.price_data = valid_data.dropna(axis=1, thresh=len(valid_data)*0.7)
        self.price_data = self.price_data.ffill()  # Forward fill missing values
        
        # Verify data
        if self.price_data.empty or len(self.price_data.columns) < 5:
            raise ValueError("Insufficient valid stock data available.")
        
        print(f"Successfully loaded data for {len(self.price_data.columns)} stocks.")
    
    def prepare_analytics(self):
        """Prepare comprehensive analytics for portfolio optimization"""
        # Calculate returns
        full_returns = np.log(self.price_data / self.price_data.shift(1)).dropna()
        
        # Use shorter window (1 year) for more relevant analysis
        self.returns = full_returns.iloc[-252:]  # Last year of trading days
        
        # Calculate volatility and other statistics
        self.daily_volatility = self.returns.std()
        self.annual_volatility = self.daily_volatility * np.sqrt(252)
        self.annual_returns = self.returns.mean() * 252
        
        # Calculate correlation and covariance
        self.correlation = self.returns.corr()
        self.covariance = self.returns.cov() * 252  # Annualized
        
        # Calculate advanced metrics
        self.calculate_advanced_metrics()
    
    def calculate_advanced_metrics(self):
        """Calculate advanced metrics for better portfolio construction"""
        # 1. Calculate momentum scores
        self.momentum_scores = self.calculate_momentum()
        
        # 2. Calculate volatility clustering
        self.volatility_clusters = self.cluster_by_volatility()
        
        # 3. Calculate drawdown resilience
        self.drawdown_metrics = self.calculate_max_drawdowns()
        
        # 4. Regime-based performance
        self.regime_performance = self.calculate_regime_performance()
        
        # 5. Calculate factor exposures
        self.factor_exposures = self.calculate_factor_exposures()
        
        # 6. Calculate Sharpe ratios
        self.sharpe_ratios = self.calculate_sharpe_ratios()
        
        # 7. Create composite score
        self.composite_scores = self.calculate_composite_scores()
        
        # 8. Calculate enhanced returns
        self.enhanced_returns = self.calculate_enhanced_returns()
        
        # 9. Select top candidates
        self.select_top_candidates()
    
    def calculate_momentum(self):
        """Calculate price momentum over specified window"""
        # Use more recent shorter-term momentum for stronger signal
        
        # Check if enough data is available for momentum calculation
        if len(self.price_data) < 63:
            # Not enough data for short term momentum, use what's available
            short_momentum = (self.price_data.iloc[-1] / self.price_data.iloc[0] - 1)
        else:
            short_momentum = (self.price_data.iloc[-1] / self.price_data.iloc[-63] - 1)  # ~3 months
        
        # Check if enough data for long-term momentum
        if len(self.price_data) < self.momentum_window:
            # Not enough data, use the same as short momentum
            long_momentum = short_momentum
        else:
            long_momentum = (self.price_data.iloc[-1] / self.price_data.iloc[-self.momentum_window] - 1)  # ~6 months
        
        # Combine short and long momentum (emphasize short-term)
        return 0.6 * short_momentum + 0.4 * long_momentum
    
    def cluster_by_volatility(self):
        """Cluster stocks by volatility patterns using a safer approach"""
        try:
            # Prepare volatility time series 
            vol_series = self.returns.rolling(window=21).std() * np.sqrt(252)
            
            # Create a simpler approach that avoids DataFrame indexing issues
            # Just use the last available volatility value for each stock as a feature
            if not vol_series.empty:
                # Get the last valid volatility for each stock
                last_volatility = vol_series.iloc[-1].fillna(vol_series.mean())
                
                # Simple clustering based on volatility level
                if len(last_volatility) >= 3:
                    # Use quantiles to define clusters
                    low_vol = last_volatility.quantile(0.33)
                    high_vol = last_volatility.quantile(0.67)
                    
                    # Assign clusters based on volatility levels
                    clusters = pd.Series(index=last_volatility.index)
                    clusters[last_volatility <= low_vol] = 0  # Low volatility cluster
                    clusters[(last_volatility > low_vol) & (last_volatility <= high_vol)] = 1  # Medium volatility
                    clusters[last_volatility > high_vol] = 2  # High volatility
                    
                    return clusters
            
            # Fallback for insufficient data
            return pd.Series(0, index=self.returns.columns)
            
        except Exception as e:
            print(f"Error in volatility clustering: {e}")
            # Return default clusters
            return pd.Series(0, index=self.returns.columns)
    
    def calculate_max_drawdowns(self):
        """Calculate maximum drawdowns to assess resilience"""
        drawdowns = {}
        
        for ticker in self.price_data.columns:
            # Calculate running maximum
            running_max = self.price_data[ticker].cummax()
            # Calculate drawdown
            drawdown = (self.price_data[ticker] / running_max - 1)
            # Get max drawdown
            drawdowns[ticker] = drawdown.min()
            
        return pd.Series(drawdowns)
    
    def calculate_regime_performance(self):
        """Analyze performance across different market regimes"""
        # Calculate simplified regime performance to avoid indexing issues
        regime_performance = {}
        
        try:
            # Calculate market returns
            market_returns = np.log(self.market_index['Close'] / self.market_index['Close'].shift(1)).dropna()
            
            # Use a simpler approach to define bull and bear markets
            if len(market_returns) >= 21:  # At least 21 days of data
                # Use 21-day moving average to identify regimes
                rolling_avg = market_returns.rolling(window=21).mean()
                bull_dates = rolling_avg[rolling_avg > 0].index
                bear_dates = rolling_avg[rolling_avg < 0].index
                
                # Analyze each ticker
                for ticker in self.returns.columns:
                    ticker_returns = self.returns[ticker]
                    
                    # Performance in bull market (shared dates between ticker_returns and bull_dates)
                    common_bull_dates = ticker_returns.index.intersection(bull_dates)
                    if len(common_bull_dates) > 0:
                        bull_performance = ticker_returns.loc[common_bull_dates].mean() * 252
                    else:
                        bull_performance = 0
                    
                    # Performance in bear market
                    common_bear_dates = ticker_returns.index.intersection(bear_dates)
                    if len(common_bear_dates) > 0:
                        bear_performance = ticker_returns.loc[common_bear_dates].mean() * 252
                    else:
                        bear_performance = 0
                    
                    # Reward stocks that perform well in bear markets
                    regime_score = bull_performance + (2.5 * bear_performance)
                    regime_performance[ticker] = regime_score
            else:
                # Not enough data for market regimes, use annual returns as proxy
                for ticker in self.returns.columns:
                    regime_performance[ticker] = self.annual_returns[ticker]
        except Exception as e:
            print(f"Error in regime performance calculation: {e}")
            # Fallback: use annual returns
            for ticker in self.returns.columns:
                regime_performance[ticker] = self.annual_returns[ticker]
        
        return pd.Series(regime_performance)
    
    def calculate_factor_exposures(self):
        """Use PCA to identify factor exposures"""
        # Safety check for empty data
        if self.returns.empty:
            return pd.Series(1, index=self.returns.columns)
            
        # Apply PCA to returns
        try:
            # Fill missing values to avoid PCA errors
            filled_returns = self.returns.fillna(0)
            
            # Determine number of components
            n_components = min(5, len(filled_returns.columns), len(filled_returns) - 1)
            
            # Apply PCA
            pca = PCA(n_components=max(1, n_components))
            pca.fit(filled_returns)
            
            # Get stock exposures to principal components
            exposures = pd.DataFrame(
                pca.components_.T, 
                index=filled_returns.columns,
                columns=[f'Factor_{i+1}' for i in range(pca.n_components_)]
            )
            
            # Calculate diversification score (lower exposure to common factors is better)
            div_score = exposures.abs().sum(axis=1)
            return 1 / div_score  # Invert so higher is better
        except Exception as e:
            # Return default values if PCA fails
            print(f"PCA calculation failed: {e}")
            return pd.Series(1, index=self.returns.columns)
    
    def calculate_sharpe_ratios(self):
        """Calculate Sharpe ratios to filter low-performing stocks"""
        excess_returns = self.annual_returns - self.risk_free_rate
        sharpe_ratios = excess_returns / self.annual_volatility
        return sharpe_ratios
    
    def calculate_composite_scores(self):
        """Create composite score combining momentum, regime performance, and drawdown"""
        # Normalize metrics to 0-1 range
        norm_momentum = normalize_series(self.momentum_scores)
        norm_regime = normalize_series(self.regime_performance)
        
        # For drawdowns, lower is better, so invert
        inverted_drawdown = -self.drawdown_metrics
        norm_drawdown = normalize_series(inverted_drawdown)
        
        # Combine into weighted composite score
        composite_scores = (
            0.45 * norm_momentum +     # Emphasize momentum
            0.35 * norm_regime +       # Regime performance
            0.20 * norm_drawdown       # Drawdown resilience
        )
        
        return composite_scores
    
    def calculate_enhanced_returns(self):
        """Calculate forward-looking enhanced returns"""
        # Start with base annualized returns (but with less weight)
        base_returns = self.annual_returns.copy()
        
        # Calculate earnings growth proxy (using price momentum as substitute)
        growth_proxy = self.momentum_scores
        
        # Normalize all factors to 0-1 range
        norm_momentum = normalize_series(self.momentum_scores)
        norm_drawdown = normalize_series(-self.drawdown_metrics)
        norm_regime = normalize_series(self.regime_performance)
        norm_factor = normalize_series(self.factor_exposures)
                      
        # Forward-looking return estimates - more heavily weighted toward predictive factors
        enhanced_returns = (
            0.25 * base_returns +            # Historical returns (less weight)
            0.30 * (growth_proxy * 0.2) +    # Growth proxy (scaled)
            0.25 * (norm_momentum * 0.25) +  # Momentum contribution
            0.10 * (norm_drawdown * 0.15) +  # Drawdown resilience
            0.10 * (norm_regime * 0.20)      # Market regime performance
        )
        
        # Ensure minimum return threshold (12%) for candidate stocks
        return enhanced_returns.clip(lower=0.12)
    
    def select_top_candidates(self):
        """Select top candidates for optimization based on composite score"""
        # Filter stocks with negative Sharpe ratios
        viable_stocks = self.sharpe_ratios[self.sharpe_ratios > 0].index.tolist()
        
        # Select top N stocks based on composite score
        n_top = min(40, len(viable_stocks))  # Select top 40 or less if not enough viable stocks
        
        # Get scores only for viable stocks
        viable_scores = self.composite_scores[viable_stocks]
        
        # Select top candidates
        self.top_candidates = viable_scores.sort_values(ascending=False).head(n_top).index.tolist()
        
        print(f"Selected {len(self.top_candidates)} top candidate stocks for optimization")
        
        # Filter relevant data to top candidates only
        self.filtered_returns = self.enhanced_returns[self.top_candidates]
        self.filtered_covariance = self.covariance.loc[self.top_candidates, self.top_candidates]

    def optimize_for_target_return(self, target_return=0.15):
        """Optimize portfolio to achieve target return with minimum risk"""
        # Use top candidates only
        n_assets = len(self.top_candidates)
        
        def portfolio_volatility(weights):
            return np.sqrt(np.dot(weights.T, np.dot(self.filtered_covariance, weights)))
        
        def portfolio_return(weights):
            return np.sum(self.filtered_returns * weights)
        
        def target_function(weights):
            # Minimize volatility
            return portfolio_volatility(weights)
        
        # Constraints
        constraints = [
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Weights sum to 1
            {'type': 'ineq', 'fun': lambda x: portfolio_return(x) - target_return}  # Min return >= target
        ]
        
        # Bounds - more flexible weights (2% to 15% per stock)
        bounds = tuple((0.02, 0.15) for _ in range(n_assets))
        
        # Initial guess - give higher weight to highest momentum stocks
        momentum_ranks = self.momentum_scores[self.top_candidates].rank()
        initial_weights = momentum_ranks / momentum_ranks.sum()
        
        try:
            # Try to achieve target return
            result = sco.minimize(
                target_function, 
                initial_weights, 
                method='SLSQP', 
                bounds=bounds, 
                constraints=constraints
            )
            
            if result['success']:
                return result['x']
            else:
                # If can't achieve target, maximize return
                return self.optimize_for_max_return()
        except Exception as e:
            print(f"Optimization for target return failed: {e}")
            # If optimization fails, use max return approach
            return self.optimize_for_max_return()
    
    def optimize_for_max_return(self, max_volatility=0.25):
        """Optimize portfolio for max return within volatility constraint"""
        # Use top candidates only
        n_assets = len(self.top_candidates)
        
        def portfolio_volatility(weights):
            return np.sqrt(np.dot(weights.T, np.dot(self.filtered_covariance, weights)))
        
        def negative_return(weights):
            # Maximize return (minimize negative return)
            return -np.sum(self.filtered_returns * weights)
        
        # Constraints
        constraints = [
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Weights sum to 1
            {'type': 'ineq', 'fun': lambda x: max_volatility - portfolio_volatility(x)}  # Max volatility
        ]
        
        # Bounds (2% to 15% per stock)
        bounds = tuple((0.02, 0.15) for _ in range(n_assets))
        
        # Initial guess - use momentum ranking
        momentum_ranks = self.momentum_scores[self.top_candidates].rank()
        initial_weights = momentum_ranks / momentum_ranks.sum()
        
        try:
            result = sco.minimize(
                negative_return, 
                initial_weights, 
                method='SLSQP', 
                bounds=bounds, 
                constraints=constraints
            )
            
            if result['success']:
                return result['x']
            else:
                # If optimization fails, use equal weights
                print("Optimization for max return failed, using equal weights")
                return np.ones(n_assets) / n_assets
        except Exception as e:
            print(f"Optimization for max return failed: {e}")
            # If optimization fails, use equal weights
            return np.ones(n_assets) / n_assets
    
    def generate_optimized_portfolio(self, target_return=0.15):
        """Generate optimized portfolio with comprehensive metrics"""
        # Optimize weights for target return (using only top candidates)
        optimal_weights = self.optimize_for_target_return(target_return)
        
        # Create results dataframe
        portfolio = pd.DataFrame({
            'Ticker': self.top_candidates,
            'Weight': optimal_weights,
            'Expected Return': self.filtered_returns,
            'Volatility': self.annual_volatility[self.top_candidates],
            'Momentum Score': self.momentum_scores[self.top_candidates],
            'Drawdown Resilience': self.drawdown_metrics[self.top_candidates],
            'Regime Performance': self.regime_performance[self.top_candidates],
            'Sharpe Ratio': self.sharpe_ratios[self.top_candidates]
        })
        
        # Sort by weight
        portfolio = portfolio.sort_values('Weight', ascending=False)
        
        # Calculate portfolio metrics
        portfolio_return = np.sum(portfolio['Weight'] * portfolio['Expected Return'])
        portfolio_volatility = np.sqrt(
            np.dot(portfolio['Weight'], np.dot(
                self.filtered_covariance, 
                portfolio['Weight']
            ))
        )
        sharpe_ratio = (portfolio_return - self.risk_free_rate) / portfolio_volatility
        
        print("\n=== ENHANCED PORTFOLIO OPTIMIZATION RESULTS ===")
        print(f"\nPortfolio Expected Return: {portfolio_return:.4f} ({portfolio_return*100:.2f}%)")
        print(f"Portfolio Volatility: {portfolio_volatility:.4f} ({portfolio_volatility*100:.2f}%)")
        print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
        
        # Show top stocks
        top_n = min(15, len(portfolio))
        print(f"\nTop {top_n} Recommended Stocks:")
        display_portfolio = portfolio.head(top_n).copy()
        display_portfolio['Weight'] = display_portfolio['Weight'] * 100  # Convert to percentage
        display_portfolio['Expected Return'] = display_portfolio['Expected Return'] * 100  # Convert to percentage
        display_portfolio['Volatility'] = display_portfolio['Volatility'] * 100  # Convert to percentage
        print(display_portfolio[['Ticker', 'Weight', 'Expected Return', 'Volatility', 'Sharpe Ratio']].to_string(float_format=lambda x: f"{x:.2f}"))
        
        return portfolio

# Helper function to normalize a series to 0-1 range
def normalize_series(series):
    """Normalize a series to 0-1 range"""
    if series.max() == series.min():
        return pd.Series(0.5, index=series.index)
    return (series - series.min()) / (series.max() - series.min() + 1e-10)

def main():
    # Create enhanced optimizer with India-appropriate parameters
    optimizer = EnhancedPortfolioOptimizer(
        tickers=nifty50_tickers,  # Using only the Nifty 50 universe
        start_date="2020-01-01",
        end_date="2025-01-01",
        risk_free_rate=0.07,  # Higher risk-free rate for India
        sentiment_factor=0.08  # Increased sentiment impact
    )
    
    # Generate optimized portfolio with target of 15% return
    portfolio = optimizer.generate_optimized_portfolio(target_return=0.15)
    
    return portfolio

if __name__ == "__main__":
    main()

Successfully loaded data for 46 stocks.
Selected 26 top candidate stocks for optimization

=== ENHANCED PORTFOLIO OPTIMIZATION RESULTS ===

Portfolio Expected Return: 0.1500 (15.00%)
Portfolio Volatility: 0.1512 (15.12%)
Sharpe Ratio: 0.5292

Top 15 Recommended Stocks:
                      Ticker  Weight  Expected Return  Volatility  Sharpe Ratio
SUNPHARMA.NS    SUNPHARMA.NS   15.00            18.27       18.95          1.92
ITC.NS                ITC.NS   12.87            12.00       18.71          0.17
DIVISLAB.NS      DIVISLAB.NS   12.54            23.15       24.97          1.83
TCS.NS                TCS.NS    7.44            12.00       20.83          0.13
PIDILITIND.NS  PIDILITIND.NS    7.02            12.00       21.74          0.24
INFY.NS              INFY.NS    3.44            12.89       22.68          0.69
ICICIBANK.NS    ICICIBANK.NS    3.13            12.94       20.64          0.87
BHARTIARTL.NS  BHARTIARTL.NS    2.57            18.66       23.26          1.83
WIPRO.NS  